<a href="https://colab.research.google.com/github/przemek-c/robotyka/blob/main/kinematyka_odwrotna_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import google.colab
%pip install roboticstoolbox-python>=1.0.2
%pip install colored==1.4.4

In [ ]:
import numpy as np
import roboticstoolbox as rtb
from spatialmath import *
from math import pi
import matplotlib.pyplot as plt
from matplotlib import cm
np.set_printoptions(linewidth=100, formatter={'float': lambda x: f"{x:8.4g}" if abs(x) > 1e-10 else f"{0:8.4g}"})

%matplotlib notebook

Rozwiązanie zagadnienia kinematyki odwrotnej, podobnie jak zagadnienia kinematyki prostej, trzeba rozpocząć od stworzenia modelu manipulatora.

In [ ]:
L1 = rtb.DHLink(d=0.645, alpha=pi/2, theta=0.0, a=0.330,)
L2 = rtb.DHLink(d=0.0, alpha=0.0, theta=0.0, a=1.35)
L3 = rtb.DHLink(d=0.0, alpha=pi/2, theta=0.0, a=0.115)
L4= rtb.DHLink(d=1.420, alpha=-pi/2, theta=0.0, a=0.0)
L5= rtb.DHLink(d=0.0, alpha=pi/2, theta=0.0, a=0.0)
L6= rtb.DHLink(d=0.215, alpha=0.0, theta=0.0, a=0.0)
robot = rtb.DHRobot([L1, L2, L3, L4, L5, L6])

W następnym kroku konieczne jest określenie położenia i orientacji końcówki manipulatora, dla której zagadnienie  ma zostać rozwiązane. Pozycję tą należny przedstawić w postaci macierzy homogenicznej. Jednym z najprostszych sposobów jest określenie macierzy translacji i rotacji. <br>
Macierz translacji stworzymy obliczając ją dla przykładu z kinematyki prostej. W ten sposób manipulator ustawi się ostatecznie w tym samym miejscu.

Rozwiązanie rozwiązania zagadnienia kinematyki odwrotnej można otrzymać wywołując na obiekcie robota metodę **ikine_LM** i podając jako argument macierz jednorodną.
```py
sol = robot.ikine_LM(T)
```
W zwracanym obiekcie *sol* zawarte są informacje o rozwiązaniu problemu, w tym współrzędne złączowe i informacja o tym czy udało się osiągnąć zadaną pozycję.

In [ ]:
i = 19
new_q = [pi/125*i, pi/2+(-pi/225*i), -pi/100*i, -pi/50*i, -pi/100*i, -pi/100*i]
T = robot.fkine(new_q)
sol = robot.ikine_LM(T)
sol

IKSolution(q=array([  0.4775,    1.306,  -0.5969,   -1.194,  -0.5969,  -0.5969]), success=True, iterations=13, searches=1, residual=5.894380647243647e-08, reason='Success')

In [ ]:
print(sol.success)
print(sol.q)

True
[  0.4775    1.306  -0.5969   -1.194  -0.5969  -0.5969]


In [ ]:
with open("wynik.txt", "w") as plik:
    plik.write(','.join(map(str,sol.q)))
    plik.write('\n')

First need path planning:
found nothing in google <br>
Need to figure out on my own: